In [91]:
#Reference : https://www.bellingcat.com/resources/2015/08/13/using-python-to-mine-common-crawl/
#Python2.7

guncontrolDomain = 'gun-control.procon.org/*'
#guncontrolDomain = 'freerepublic.com'
gunsenseDomain = 'gunviolencearchive.org/*' 
nraDomain = 'nraila.org/articles/*'
#firearmsDomain = 'usatoday.com/story/*'

    

In [92]:
import requests
import argparse
import time
import json
import StringIO
import gzip
import csv
import codecs

from bs4 import BeautifulSoup

import sys
reload(sys)
sys.setdefaultencoding('utf8')


In [93]:
#ap = argparse.ArgumentParser()
#ap.add_argument("-d","--domain",required=True,help="The domain to target ie. cnn.com")
#args = vars(ap.parse_args())

#domain = guncontrolDomain
#domain = nraDomain
domain = gunsenseDomain

# list of indices
index_list = ["2019-04","2019-09","2019-11","2019-14","2019-16"]

In [86]:
def search_domain(domain):

    record_list = []
    
    print "[*] Trying target domain: %s" % domain
    
    for index in index_list:
        
        print "[*] Trying index %s" % index
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&output=json" % domain
        print(cc_url)
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print "[*] Added %d results." % len(records)
            
    
    print "[*] Found a total of %d hits." % len(record_list)
    
    return record_list 

In [87]:
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    
    raw_data = StringIO.StringIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # WARC Response
    data = f.read()
    
    response = ""
    
    if len(data):
        try:
            warc, header, response = data.strip().split('\r\n\r\n', 2)
        except:
            pass
            
    return response

In [88]:
nra = []
def getDataFromHtmlContent(html_content,keywords):
    #print(html_content)
    check = False
    soup = BeautifulSoup(html_content)
    for script in soup(["script", "style"]):
        script.extract()
    
    text = soup.get_text()
    
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    #print(text[:10])
    #words = text.split()
    for word in keywords:
        if(word in text):
            check = True
            break
            
    if(check):
        filename='data/cc/gunsense.txt'
        f=open(filename,'a')
        f.write(text+"\n")
        nra.append(1)
        f.close()
    
    # break into lines and remove leading and trailing space on each
    #lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    #chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #text = '\n'.join(chunk for chunk in chunks if chunk)
    #print(text)
    #filename='data/cc/'+searchKey+'/'+searchKey+str(n)+'.txt'
    #f=open(filename,'a')
    #f.write(soup.find_all('body'))
    #print(soup)
    
    
    #soup.prettify()
    
    #print(soup.find_all('p'))
    #filename='data/cc/'+searchKey+'/'+searchKey+str(n)+'.txt'
    #f=open(filename,'a')
    #f.write(html_content)
    #for j in range((len(soup.find_all('p')))-3):
        #if (searchKey in soup.find_all('p')[j].get_text()):
    #    f.write(soup.find_all('p')[j].get_text())
    
    #f.close() 
    #return html_content


In [94]:
record_list = search_domain(domain)
link_list   = []



In [95]:
record_list[:1]

[{u'digest': u'3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
  u'filename': u'crawl-data/CC-MAIN-2019-04/segments/1547583705737.21/crawldiagnostics/CC-MAIN-20190120102853-20190120124853-00490.warc.gz',
  u'length': u'595',
  u'mime': u'unk',
  u'mime-detected': u'application/octet-stream',
  u'offset': u'10008244',
  u'status': u'301',
  u'timestamp': u'20190120113222',
  u'url': u'http://www.gunviolencearchive.org/',
  u'urlkey': u'org,gunviolencearchive)/'}]

In [97]:
keywords = ["gun control","gun voilence","nra","gun laws","firearms","second amendment","gun"]
for record in record_list:
    
    html_content = download_page(record)
    getDataFromHtmlContent(html_content,keywords)
   
    #soup = BeautifulSoup(html_content)
    #for script in soup(["script", "style"]):
    #    script.extract()
    
    #text = soup.get_text()
    
    #lines = (line.strip() for line in text.splitlines())
    #chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    #text = '\n'.join(chunk for chunk in chunks if chunk)
    #print(text)
        
    #print(soup.find("h1").get_text())
    
    #print(soup.findAll("div", {"class": "asset-double-wide"})[0].prettify())
    
    
    
    #print(soup.find_all('p'))
#     print ("[*] Retrieved %d bytes for %s" , (len(html_content),record['url']))
#     print(record['url'])
    
#     data = getDataFromHtmlContent(html_content,n,'gunman')
#     n=n+1
#     print(data)



In [98]:
("Total articles in common crawl data:" ,len(nra))

('Total articles in common crawl data:', 722)